In [2]:
import pandas as pd
import requests
import calendar

### DATOS DE ENTRADA:

ciudad = 'Sevilla'
mes = 'Jul'
potencia = 3.5
c_punta = 188.39
c_llano = 233.13
c_valle = 355.02
consumo = [c_punta,c_llano,c_valle]


In [4]:

### DATOS HORARIOS POR PROVINCIA

# Con esta función sacamos un df con los datos de horas de luz de la provincia especificada.

def leer_datos_horarios(ciudad):    
    df = pd.read_csv("datos_provincias.csv")
    provincias = ['Almeria','Cadiz','Cordoba','Granada','Huelva','Jaen',
            'Malaga','Sevilla','Huesca','Zaragoza','Teruel','Oviedo',
            'Barcelona','Tarragona','Girona','Lleida','Coruna','Lugo',
            'Ourense','Pontevedra','Santander','Albacete','Ciudad',
            'Cuenca','Guadalajara','Toledo','Avila','Burgos','Leon','Salamanca',
            'Segovia','Valladolid','Zamora','Alicante','Valencia','Castellon',
            'Palma','Palmas','Santa','Logrono','Bilbao','Donostia','Vitoria',
            'Caceres','Badajoz','Madrid','Pamplona','Murcia']
    nuevo_df = df[df['Ciudad'] == ciudad]
    return nuevo_df



In [5]:


## CÁLCULO PLACAS SOLARES

# Lo primero es prorratear los datos de consumo valle para que no distinga laborables de fines de semana:

def prorrateo_consumo(mesecito,consumito):
    
    # Indicamos el cambio a datos numéricos
    year = 2024
    mes = {
    'Ene': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12}
    month = mes[mesecito]

    # Definimos los períodos para un día laborable
    discriminacion = {
        "00": 'valle', "01": 'valle', "02": 'valle', "03": 'valle', 
        "04": 'valle', "05": 'valle', "06": 'valle', "07": 'valle',
        "08": 'llano', "09": 'llano', "10": 'punta', "11": 'punta',
        "12": 'punta', "13": 'punta', "14": 'llano', "15": 'llano',
        "16": 'llano', "17": 'llano', "18": 'punta', "19": 'punta',
        "20": 'punta', "21": 'punta', "22": 'llano', "23": 'llano',
    }

    # Calcula horas diarias por tipo para días laborables
    horas_llano_lab = sum(1 for h in discriminacion if discriminacion[h] == 'llano')
    horas_punta_lab = sum(1 for h in discriminacion if discriminacion[h] == 'punta')

    # Obtén el número total de días en el mes
    num_dias_mes = calendar.monthrange(year, month)[1]

    # Contar cuántos sábados y domingos hay en ese mes
    dias_finde = sum(1 for d in range(1, num_dias_mes + 1) if calendar.weekday(year, month, d) >= 5)
    dias_laborables = num_dias_mes - dias_finde
    
    # Calcular horas totales para ese mes
    horas_llano_totales = horas_llano_lab * dias_laborables
    horas_punta_totales = horas_punta_lab * dias_laborables
    
    #Calculamos los consumos prorrateados, obviando la diferencia de laborable o no
    c_p = consumito[0]*8/horas_punta_totales
    c_l = consumito[1]*8/horas_llano_totales
    c_v = (sum(consumito) - c_p*num_dias_mes - c_l*num_dias_mes)/num_dias_mes

    c_totales = [c_p*num_dias_mes,c_l*num_dias_mes,c_v*num_dias_mes]
    return c_totales


# Calculamos el numero de placas estimado:

# Prorrateamos el consumo para el mes
consumo_total = prorrateo_consumo(mes,consumo)

# Sacamos los datos de la ciudad especificada
df_ciudad = leer_datos_horarios(ciudad)

factor_solar = 0.8 # Durante el ocaso y el amanecer, las horas de sol no son tan efectivas, por lo que
                # se introduce este factor para sere conservativos en el cálculo
pot_placa = 0.455 #kWp
dias = {
'Ene': 31,
'Feb': 29,
'Mar': 31,
'Apr': 30,
'May': 31,
'Jun': 30,
'Jul': 31,
'Aug': 31,
'Sep': 30,
'Oct': 31,
'Nov': 30,
'Dec': 31}

# Crear el diccionario para almacenar las sumas mensuales
meses = df_ciudad.columns.difference(['Dia', 'Ciudad'])
horas_sol = {mes: df_ciudad[mes].sum()*factor_solar for mes in meses}
porcen_sol = {mes: round(df_ciudad[mes].sum()*factor_solar/(dias[mes]*24),2) for mes in meses}
# Aproximadamente entre el 40% y el 50% de luz al día. 

# El consumo anual de energía por parte del consumidor debe ser igual o mayor 
# al 80 %  de la energía anual generada por la instalación.

# c_total >= e_generada *0.8

c_total = sum(consumo_total)
n_placas = int(c_total / (pot_placa * 0.7 * horas_sol[mes] * 0.8))
if n_placas <2:
    print('''Tu consumo es muy bajo para poder beneficiarte de una instalación de placas solares.
        ¡Gracias por ser un consumidor eficiente!''')
else:
    print(f'Con los datos de consumo suministrados, ¡Te podría interesar instalar hasta {n_placas} placas!')
    print('''Ten en cuenta que la mejor estimación del número de placas se realiza con el consumo en verano, 
        además de ser donde te beneficiarás del mayor ahorro.''')
 


Con los datos de consumo suministrados, ¡Te podría interesar instalar hasta 8 placas!
Ten en cuenta que la mejor estimación del número de placas se realiza con el consumo en verano, 
        además de ser donde te beneficiarás del mayor ahorro.


In [6]:
## AHORRO PLACAS

e_generada = n_placas * pot_placa * 0.7 * horas_sol[mes]

# La energia generada no afecta al periodo valle y sólo afecta al 50% del periodo llano:

datos_consumo= {'Punta' : max(0,consumo_total[0]-e_generada*2/3),
                'Llano' : max(0,consumo_total[1]-e_generada*1/3),
                'Valle' : consumo_total[2], 
                'Excedentes': min(0,consumo_total[0]-e_generada*2/3) + min(0,consumo_total[1]-e_generada*1/3),
                'Dias': dias[mes],
                'Potencia': potencia }

## TARIFA SOLAR

# Una vez conocidos los consumos con las placas, obtenemos el precio con las tarifas solares:

df_solar = pd.read_csv("tarifas_solar.csv")

datos_consumo

def calcular_mejor_tarifa(consumo):
    # Parámetros
        
    iva = 1.21
    bono_social = 0.006282
    impuesto = 3.8 / 100
    equipos = 0.82

    df = pd.read_csv("tarifas_solar.csv")
    # Crear un DataFrame con los resultados
    resultado_df = df[['Empresa', 'Tarifa']].copy()
    
    consumo_total = (df['Punta'] * consumo['Punta'] +
               df['Llano'] * consumo['Llano'] +
               df['Valle'] * consumo['Valle'])
    resultado_df['Consumo'] = consumo_total
    excedentes_total =  df['Excedentes'] * consumo['Excedentes']
    resultado_df['Excedentes'] = excedentes_total
    potencia_total = (df['P1']+df['P3'])*consumo['Potencia']*consumo['Dias']
    resultado_df['Potencia'] = potencia_total
    precio_bat = ((potencia_total + bono_social * consumo['Dias'] + consumo_total) * (1 + impuesto) +
                  equipos + df["Bateria"] * consumo['Dias']/30) * iva + excedentes_total
    resultado_df['Precio con Bateria'] = precio_bat
    consumo_total_nobat = (consumo_total + excedentes_total).clip(lower=0)
    precio_nobat = ((potencia_total + bono_social * consumo['Dias'] + consumo_total_nobat) * (1 + impuesto) +
                  equipos) * iva
    resultado_df['Precio sin Bateria'] = precio_nobat

    # Crear columna precio con las condiciones reales de cada compañía

    resultado_df['Precio'] = resultado_df.apply(
    lambda row: row['Precio sin Bateria'] if row['Empresa'] == 'Naturgy' else row['Precio con Bateria'], axis=1)
    
    df_sorted = resultado_df.sort_values(by='Precio', ascending=True)
    mejor_tarifa = df_sorted.iloc[0]

    print("\nComparativa de tarifas:")
    for _, tarifa in df_sorted.iterrows():  # Usamos iterrows para iterar sobre las filas
        print(f"{tarifa['Empresa']} - {tarifa['Tarifa']}: {tarifa['Precio']:.2f} €")

    print(f"\nLa tarifa más económica es la de {mejor_tarifa['Empresa']} - {mejor_tarifa['Tarifa']} con un precio de {mejor_tarifa['Precio']:.2f} €.")

    return 

# Llamar a la función
tarifas = calcular_mejor_tarifa(datos_consumo)
tarifas


Comparativa de tarifas:
Naturgy - Solar: 20.00 €
Octopus - Octopus Solar: 20.41 €
Repsol - Tarifa Placa Solar: 32.12 €

La tarifa más económica es la de Naturgy - Solar con un precio de 20.00 €.
